### Getting Data

In [ ]:
transformation  = transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize((0.1307,),(0.3081))])

train_dataset = datasets.MNIST('data/', train = True, transform=transformation)
test_dataset = datasets.MNIST('data/', train = False, transform=transformation)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size= batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size= batch_size, shuffle=True)

In [ ]:
def plot_img(image):
    image = image.numpy()[0]
    mean = 0.1307
    std = 0.3081
    image = ((mean * image) + std)
    plt.imshow(image)
    
sample = next(iter(train_loader))
plot_img(sample_data()[1])
plot_img(sample_data()[2])

### Architecture

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
        
    def forward(self,x):
        x =F.relu(F.max_pool2d(self.conv1(x),2))
        x =F.relu(F.max_pool2d(self.conv2_drop(x),2))
        x = x.view(-1,320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training = self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x)

### view

we are using a two dimensional convolution that takes a matrix of numbers as input and outputs another matrix of numbers. To apply a linear layer, we need to flatten the matrix which is a tensor of two dimensions to a vector of one dimension 

x.view(-1, 320)

### Training

In [ ]:
def fit(epoch, model, data_loader, phase = 'training', valotile=False):
    if (phase == 'training'):
        model.train()
    if (phase == 'validation'):
        model.eval()
        valotile = True
    
    running_loss = 0
    running_correct = 0
    
    for batch_idx, (data,target) in enumerate(data_loader):
        if is_cuda:
            data,target = data.cuda(), target.cuda()
        
        data, target = Variable(data, valotile), Variable(target)
        
        if phase == 'training':
            optimize.zero_grad()
        
        output = model(data)
        loss = F.nll_loss(output, target)
        
        running_loss += F.nll_loss(output, target, size_average = False ).data[0]
        
        preds= output.data.max(dim =1, keep_dim = True)[1]
        
        running_corect += preds.eq(target.data.view_as(preds)).cpu().sum()
        
        if phase == 'training':
            loss.backward()
            optmizer.step()
    
    